In [1]:
import gymnasium as gym
import gymnasium_2048 
import numpy as np
from tqdm import trange

from simulations.base_agents import DQNAgentWrapper
from simulations.interpretable_agents import ViperAgent

c:\Users\Polina\Desktop\envs\project_2048\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ENV_ID = "gymnasium_2048/TwentyFortyEight-v0"

## VIPER agent

In [3]:
oracle = DQNAgentWrapper("env_2048/save/best/dqn_7710.pkl")

In [4]:
env = gym.make(ENV_ID) 

viper = ViperAgent(
    env,
    oracle,
    num_iters=15,        
    num_trajs=80,        
    max_depth=8,
    random_state=42
)

VIPER it:   0%|          | 0/15 [00:00<?, ?it/s]c:\Users\Polina\Desktop\envs\project_2048\lib\site-packages\gymnasium_2048\envs\twenty_forty_eight.py:250: RuntimeWarning: overflow encountered in scalar add
  self.total_score += self.step_score
VIPER it:   0%|          | 0/15 [05:59<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# # Инициализация VIPER-агента
# ORACLE_PATH = 'drl2048/trained_architectures/convdqn_torch_300_net.pt'
# env = gym.make(ENV_ID)
# viper = ViperAgent(
#     env=env,
#     oracle_path=ORACLE_PATH,
#     num_rollouts=500,      # Количество эпизодов для сбора данных
#     max_depth=12,           # Максимальная глубина дерева
#     batch_size=128,         # Размер батча для обучения
#     gamma=0.95              # Коэффициент дисконтирования
# )

In [ ]:
# viper.collect_data()   # tqdm-полоса «VIPER Rollouts»
# viper.train()          # ещё один tqdm «Preparing training data»

In [ ]:
# # создаём псевдонимы для старых импорт-путей внутри pickle
# sys.modules['modules'] = real_architectures.__package__ and __import__('drl2048.modules', fromlist=['']) or real_architectures
# sys.modules['modules.architectures'] = real_architectures

# class ViperAgent(BaseAgent):
#     """VIPER agent с извлечением стратегии через имитационное обучение"""
    
#     def __init__(
#         self,
#         env,
#         oracle_path,
#         num_rollouts=500,
#         max_depth=10,
#         batch_size=64,
#         gamma=0.99
#     ):
#         super().__init__(env)
        
#         # Параметры обучения - define these FIRST
#         self.num_rollouts = num_rollouts
#         self.max_depth = max_depth
#         self.batch_size = batch_size
#         self.gamma = gamma
        
#         # Then load the oracle model
#         self.oracle = self._load_oracle(oracle_path)
#         self.oracle.eval()
        
#         # Буферы данных
#         self.dataset = deque(maxlen=100000)
#         self.tree = DecisionTreeClassifier(max_depth=self.max_depth)
        
#         # Конвертер состояний
#         self.tile_values = [2**i for i in range(1, 16)]
        
#     def _load_oracle(
#         self, 
#         path: str, 
#         agent_params={
#             'gamma': 0.995,
#             'replay_memory_size': 10000,
#             'batch_size': 384,
#             'eps_start': 0.8,
#             'eps_end': 0.02, 
#             'eps_decay': 1000,             
#             'tau': 5e-3,
#             'kind_action': "entropy",
#             'lr': 1e-3
#         }, 
#         model_params={'l2_regularization': 3e-4}
#     ) -> nn.Module:
#         return ConvDQNAgentWrapper(model_path=path, agent_params=agent_params, model_params=model_params).model
    
#     def _state_to_tensor(self, board: np.ndarray):
#         """board 4×4 -> tensor 1×12×4×4, без Python-циклов"""
#         # board → экспоненты (-1, 0 … 11)  | -1 = пустая клетка
#         exps = np.where(board > 0, np.clip(np.log2(board).astype(int) - 1, 0, 11), -1)
#         # one-hot: (4,4,12)
#         onehot = np.eye(12, dtype=np.float32)[exps.clip(0)]       # пустые ячейки = канал 0
#         tensor = torch.from_numpy(onehot.transpose(2,0,1))        # → (12,4,4)
#         return tensor.unsqueeze(0).to(self.device)
    
#     def collect_data(self):
#         """Сбор данных траекторий oracle-сети для VIPER."""
#         for _ in tqdm(range(self.num_rollouts), desc="VIPER Rollouts"):
#             obs, _ = self.env.reset()
#             done = False
#             while not done:
#                 board = get_tile_value(obs)                 # [НОВОЕ]
#                 state_tensor = self._state_to_tensor(board) # (а не raw obs)
#                 with torch.no_grad():
#                     q_values = self.oracle(state_tensor).cpu().numpy()[0]

#                 # advantage gap
#                 advantage = np.partition(q_values, -2)[-1] - np.partition(q_values, -2)[-2]

#                 # interpretable features от **numeric** board
#                 features = extract_features(board)

#                 self.dataset.append({
#                     "features": list(features.values()),
#                     "action": int(q_values.argmax()),
#                     "weight": advantage ** 2,
#                 })

#                 obs, _, done, _, _ = self.env.step(int(q_values.argmax()))

#     def _state_to_tensor(self, state: np.ndarray):
#         """Convert raw env obs (4×4×17) **или** numeric board (4×4) → 1×12×4×4 binary tensor."""
#         if state.ndim == 3:                       # [НОВОЕ] пришёл raw-тензор из среды
#             state = get_tile_value(state)         # -> (4, 4) с числами 0/2/4/…

#         tensor = torch.zeros(12, 4, 4, device=device)
#         for i in range(4):
#             for j in range(4):
#                 val = state[i, j]
#                 if val == 0:
#                     continue
#                 exp = min(int(np.log2(val)) - 1, 11)   # всё, что ≥4096, уезжает в последний канал
#                 tensor[exp, i, j] = 1
#         return tensor.unsqueeze(0)
    
#     def train(self):
#         """Обучение дерева решений с учётом весов + индикатор прогресса"""
#         if not self.dataset:
#             raise ValueError("Сначала выполните сбор данных через collect_data()")

#         # —–– Подготовка массивов с progress-bar –––
#         X, y, weights = [], [], []
#         for sample in tqdm(self.dataset,
#                            desc="Preparing training data",
#                            unit="samples"):
#             X.append(sample['features'])
#             y.append(sample['action'])
#             weights.append(sample['weight'])

#         X = np.asarray(X, dtype=np.float32)
#         y = np.asarray(y, dtype=np.int64)
#         weights = np.asarray(weights, dtype=np.float32)

#         # —–– Обучение дерева (одиночным шагом) –––
#         # tqdm.write выводит сообщение, не «ломая» полосу прогресса
#         tqdm.write("Fitting decision tree…")
#         self.tree.fit(X, y, sample_weight=weights)
#         tqdm.write("Decision tree training complete ✅")
    
#     def act(self, observation):
#         """Действие по обученному дереву решений."""
#         board = get_tile_value(observation) if observation.ndim == 3 else observation
#         feats = extract_features(board)
#         return int(self.tree.predict([list(feats.values())])[0])

In [ ]:
# class SindyAgent(BaseAgent):
#     """
#     Интерпретируемая SINDy-политика:
#     1) собирает rollout’ы у обученного DQN-оракула,
#     2) извлекает фичи φ(s) = extract_features(board),
#     3) строит полиномиальную библиотеку признаков Poly(φ),
#     4) обучает 4 LASSO-регрессора Q̂_a(φ) для каждого действия a.
#     При act выбирает a = argmax_a Q̂_a(φ(s)), игнорируя нелегальные ходы.
#     """
#     def __init__(
#         self,
#         env,
#         oracle_path: str,
#         num_rollouts: int = 200,
#         poly_order: int = 2,
#         alpha: float = 0.01
#     ):
#         super().__init__(env)
#         # 1) оракул
#         self.oracle = ConvDQNAgentWrapper(oracle_path)
#         self.num_rollouts = num_rollouts
#         self.poly_order = poly_order
#         self.alpha = alpha
#         # 2) сбор данных + обучение
#         X, Y = self._collect_data()
#         self._fit_sindy_policy(X, Y)

#     def _collect_data(self):
#         """Запускает несколько игр у oracle, собирает φ(s) и истинные Q(s,a)."""
#         X_list, Y_list = [], []
#         for _ in tqdm(range(self.num_rollouts)):
#             obs, _ = self.env.reset()
#             done = False
#             while not done:
#                 board = get_tile_value(obs)
#                 φ = extract_features(board)
#                 # Q-ценности из оракула
#                 state_t = ConvDQNAgentWrapper._encode_board(board).unsqueeze(0).to(device)
#                 with torch.no_grad():
#                     q_vals = self.oracle.agent.model(state_t).cpu().numpy().ravel()
#                 X_list.append(φ)
#                 Y_list.append(q_vals)
#                 # шаг по оракулу
#                 a = int(np.argmax(q_vals))
#                 obs, _, terminated, truncated, _ = self.env.step(a)
#                 done = terminated or truncated
#         X = np.vstack(X_list)               # shape=(N, n_features)
#         Y = np.vstack(Y_list)               # shape=(N, 4)
#         return X, Y

#     def _fit_sindy_policy(self, X: np.ndarray, Y: np.ndarray):
#         """Строим полиномиальную библиотеку и обучаем 4 LASSO-регрессора."""
#         poly = PolynomialFeatures(degree=self.poly_order, include_bias=False)
#         Xp = poly.fit_transform(X)
#         models = []
#         for a in range(Y.shape[1]):
#             lasso = Lasso(alpha=self.alpha, max_iter=10000)
#             lasso.fit(Xp, Y[:, a])
#             models.append(lasso)
#         self.poly = poly
#         self.models = models
#         # для интерпретации можно сохранить имена признаков:
#         self.term_names = poly.get_feature_names_out(feature_names)

#     def act(self, obs: np.ndarray) -> int:
#         board = get_tile_value(obs)
#         φ = extract_features(board).reshape(1, -1)
#         Xp = self.poly.transform(φ)
#         q_pred = np.array([m.predict(Xp)[0] for m in self.models])
#         # запрещаем нелегальные ходы
#         legal = ConvDQNAgentWrapper._legal_moves(board)
#         q_pred[~legal] = -np.inf
#         return int(np.argmax(q_pred))
    
#     def _action_name(self, idx: int) -> str:
#         """Удобное текстовое имя действия по индексу 0…3."""
#         return ["up", "down", "left", "right"][idx]
    
#     def get_symbolic_policy(self, threshold: float = 1e-5) -> dict:
#         """
#         Возвращает разреженное представление политики в виде:
#         {
#             'up':   { 'empty_ratio':  +0.42, 'max_tile_ratio': -0.37, ... },
#             'down': { ... },
#             ...
#         }
#         Включаются только коэффициенты |coef| >= threshold.
#         """
#         if not hasattr(self, "models"):
#             raise RuntimeError("SindyAgent ещё не обучен (models отсутствуют).")

#         policy = {}
#         for a, model in enumerate(self.models):
#             nonzero = np.abs(model.coef_) >= threshold
#             terms = {
#                 term: float(coef)
#                 for coef, term in zip(model.coef_[nonzero], self.term_names[nonzero])
#             }
#             policy[self._action_name(a)] = dict(sorted(terms.items(), key=lambda kv: -abs(kv[1])))
#         return policy
